In [8]:
import pandas as pd
import numpy as np
import os
import json

# 读取数据
root_path = "/workspace/med_proj"
file_path = f"{root_path}/data/数据汇总.csv"
data = pd.read_csv(file_path, index_col=None, header=0)
# data = pd.read_excel(file_path)

id2cluster = pd.read_csv(f"{root_path}/data/48h/result.csv", index_col=None, header=0)
with open(f"{root_path}/results/dataprocess/48h/name_id_mapping.json", "r", encoding="utf-8") as f:
    name2id = json.load(f)
id2name = {v: k for k, v in name2id.items()}

name2cluster = {}
for index, row in id2cluster.iterrows():
    name2cluster[id2name[row['ID']]] = row['cluster']

In [9]:
data.columns

Index(['name', '白细胞', 'CRP', 'TNF', 'PTC', 'il6', 'il10', '性别', '年龄', 'SOFA',
       'APACHE', 'prognosis', '机械通气', '住院时长', '是否二感', '慢性病个数', '入院感染部位',
       '新感部位', 'BMI', '血红蛋白', '血小板', '总胆', '间胆', '白蛋白', 'AST', 'ALT', '肌酐',
       'CRRT', '深静脉置管', '入量', '出量', '血管活性药物', '慢性心脏疾病', '慢性肝脏疾病', '慢性肺疾病',
       '慢性肾脏疾病', '合并症', 'cluster', '感染部位拼接', '入量-出量', '高血压', '糖尿病', '肿瘤'],
      dtype='object')

In [2]:
with open(f"{root_path}/results/anova/48h/name2cluster.txt", "w", encoding="utf-8") as f:
    for name, cluster in name2cluster.items():
        f.write(f"{name}, {cluster}\n")

In [17]:
root_path = "/workspace/med_proj"
file_path = f"{root_path}/data/数据汇总.csv"
data = pd.read_csv(file_path)
data["入院感染部位"].value_counts()

入院感染部位
0    274
1     39
Name: count, dtype: int64

In [13]:
data.columns

Index(['name', '白细胞', 'CRP', 'TNF', 'PTC', 'il6', 'il10', '性别', '年龄', 'SOFA',
       'APACHE', 'prognosis', '机械通气', '住院时长', '是否二感', '慢性病个数', '入院感染部位',
       '新感部位', 'BMI', '血红蛋白', '血小板', '总胆', '间胆', '白蛋白', 'AST', 'ALT', '肌酐',
       'CRRT', '深静脉置管', '入量', '出量', '血管活性药物', '慢性心脏疾病', '慢性肝脏疾病', '慢性肺疾病',
       '慢性肾脏疾病', '合并症', 'cluster', '感染部位拼接', '入量-出量', '高血压', '糖尿病', '肿瘤'],
      dtype='object')

In [ ]:
df_filtered = data[data['name'].isin(name2id.keys())]

# 将 name2id 中的对应ID添加到新列中
df_filtered['cluster'] = df_filtered['name'].map(name2cluster)
df_filtered.rename(columns={'姓名': 'name'}, inplace=True)
df_filtered['性别'] = df_filtered['性别'].replace({'男': 1, '女': 0})
# df_filtered['转归'] = df_filtered['转归'].replace({'存活': 0,'死亡': 1, '好转':0, "未愈": 1, "其他":0})
df_filtered['机械通气'] = df_filtered['机械通气'].replace({'无': 0, '有': 1})
df_filtered['是否二感'] = df_filtered['是否二感'].replace({'否': 0, '是': 1, '疑似': 0})
df_filtered['感染部位拼接'] = df_filtered['入院感染部位'] + df_filtered['新感部位']
df_filtered['入量-出量'] = df_filtered['入量'] - df_filtered['出量']

df_filtered['慢性病个数'] = df_filtered['慢性病个数'].replace({0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1})

df_filtered['感染部位拼接'] = df_filtered['感染部位拼接'].apply(lambda x: 1 if '血液' in x else 0)
# df_filtered['入院感染部位'] = df_filtered['入院感染部位'].apply(lambda x: 1 if '血液' in x else 0)
df_filtered['新感部位'] = df_filtered['新感部位'].apply(lambda x: 1 if '血液' in x else 0)

df_filtered['CRRT'] = df_filtered['CRRT'].replace({'无': 0, '是': 1})
df_filtered['深静脉置管'] = df_filtered['深静脉置管'].replace({'无': 0, '是': 1})
df_filtered['血管活性药物'] = df_filtered['血管活性药物'].replace({'无': 0, '是': 1})

df_filtered['慢性心脏疾病'] = df_filtered['慢性心脏疾病'].apply(lambda x: 1 if '心力衰竭' in x else 0)
df_filtered['慢性肝脏疾病'] = df_filtered['慢性肝脏疾病'].apply(lambda x: 0 if '无' in x else 1)
df_filtered['慢性肺疾病'] = df_filtered['慢性肺疾病'].apply(lambda x: 1 if '慢阻肺' in x else 0)
df_filtered['慢性肾脏疾病'] = df_filtered['慢性肾脏疾病'].apply(lambda x: 0 if '无' in x else 1)


df_filtered['高血压'] = df_filtered['合并症'].apply(lambda x: 1 if '高血压' in x else 0)
df_filtered['糖尿病'] = df_filtered['合并症'].apply(lambda x: 1 if '糖尿病' in x else 0)
df_filtered['肿瘤'] = df_filtered['合并症'].apply(lambda x: 1 if '肿瘤' in x else 0)

df_filtered['合并症'] = df_filtered['合并症'].apply(lambda x: 1 if '无' in x else 0)


TypeError: argument of type 'int' is not iterable

In [5]:
df_filtered.to_csv(f'{root_path}/results/anova/48h/cluster_feature_adjust.csv', index=False, encoding='utf-8-sig')

In [18]:
root_path = "/workspace/med_proj"
file_path = f"{root_path}/data/数据汇总.csv"
data = pd.read_csv(file_path)

In [19]:
data.columns

Index(['name', '白细胞', 'CRP', 'TNF', 'PTC', 'il6', 'il10', '性别', '年龄', 'SOFA',
       'APACHE', 'prognosis', '机械通气', '住院时长', '是否二感', '慢性病个数', '入院感染部位',
       '新感部位', 'BMI', '血红蛋白', '血小板', '总胆', '间胆', '白蛋白', 'AST', 'ALT', '肌酐',
       'CRRT', '深静脉置管', '入量', '出量', '血管活性药物', '慢性心脏疾病', '慢性肝脏疾病', '慢性肺疾病',
       '慢性肾脏疾病', '合并症', 'cluster', '感染部位拼接', '入量-出量', '高血压', '糖尿病', '肿瘤'],
      dtype='object')

### 输出 OR 值以及置信区间

In [20]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Assuming df_filtered is your DataFrame
data = data

# The column to compare against all other columns
reference_column = 'prognosis'

# Extracting columns for Logistic Regression, excluding 'name', 'cluster' and '转归' itself
logistic_columns = data.columns.drop(['name', 'cluster', reference_column])

# Dictionary to hold Logistic Regression results
logistic_results = {}

# Function to calculate the odds ratio, 95% confidence intervals, and p-values
def calculate_odds_ratios(result):
    params = result.params
    p_values = result.pvalues  # Get the p-values
    conf = result.conf_int()
    conf['OR'] = np.exp(params)  # Calculating the odds ratio
    conf.columns = ['2.5%', '97.5%', 'OR']  # Renaming columns
    # Applying exp() to 95% confidence intervals
    conf[['2.5%', '97.5%']] = np.exp(conf[['2.5%', '97.5%']])
    # Adding p-values
    conf['P-value'] = p_values
    return conf

# Performing Logistic Regression for each column and storing the results
for column in logistic_columns:
    # Ensure the independent variable is numeric and does not contain missing values
    X = sm.add_constant(data[column].astype(float))  # Add constant for intercept
    
    # Define the dependent variable (binary)
    y = data[reference_column].astype(int)  # Ensure the dependent variable is binary
    
    # Fit the Logistic Regression model
    logit_model = sm.Logit(y, X).fit(disp=0)
    
    # Calculate OR, confidence intervals, and p-values
    or_ci = calculate_odds_ratios(logit_model)
    
    # Storing OR, CI, and p-values for each feature
    logistic_results[column] = or_ci.loc[column]

# Convert Logistic Regression results to a DataFrame
logistic_df = pd.DataFrame(logistic_results).T

# Define root_path if not defined

# Saving the results to CSV files
os.makedirs(f"{root_path}/results/logistic/48h", exist_ok=True)  # Create directory if not exists
logistic_df.to_csv(f'{root_path}/results/logistic/48h/logistic_vs_zhuan_gui.csv', encoding="utf-8-sig")



### 使用方差分析得到结结果

In [21]:
from scipy.stats import f_oneway
import numpy as np

data = df_filtered
# Extracting columns for ANOVA, excluding 'name' and 'cluster' columns
anova_columns = data.columns.drop(['name', 'cluster'])

# Dictionary to hold ANOVA results
anova_results = {}

# Performing ANOVA for each column and storing the results
for column in anova_columns:
    groups = data.groupby('cluster')[column].apply(list)
    f_val, p_val = f_oneway(*groups)
    anova_results[column] = {'F-value': f_val, 'p-value': p_val}

# Convert ANOVA results to a DataFrame
anova_df = pd.DataFrame(anova_results).T

# Calculating mean values for each column per cluster
mean_values = data.groupby('cluster')[anova_columns].mean()

# Calculating median values for each column per cluster
median_values = data.groupby('cluster')[anova_columns].median()

std_values = data.groupby('cluster')[anova_columns].std()

# Calculating count of each column per cluster
group_counts = data.groupby('cluster').size().reset_index(name='count')

q1 = data.groupby('cluster')[anova_columns].quantile(0.25)
q3 = data.groupby('cluster')[anova_columns].quantile(0.75)
iqr_values = q3 - q1

anova_df.to_csv(f'{root_path}/results/anova/48h/anova.csv', encoding="utf-8-sig")
mean_values.to_csv(f'{root_path}/results/anova/48h/mean_values.csv', encoding="utf-8-sig")
median_values.to_csv(f'{root_path}/results/anova/48h/median_values.csv', encoding="utf-8-sig")
std_values.to_csv(f'{root_path}/results/anova/48h/std_values.csv', encoding='utf-8-sig')
group_counts.to_csv(f'{root_path}/results/anova/48h/group_counts.csv', encoding="utf-8-sig")
q1.to_csv(f'{root_path}/results/anova/48h/q1_values.csv', encoding='utf-8-sig')  # Export Q1
q3.to_csv(f'{root_path}/results/anova/48h/q3_values.csv', encoding='utf-8-sig')
iqr_values.to_csv(f'{root_path}/results/anova/48h/iqr_values.csv', encoding='utf-8-sig')


#### 比较其中一组和其他三组的差异

In [22]:
from scipy.stats import f_oneway
import pandas as pd

# 假设 df_filtered 是你的数据框
data = df_filtered

# 初始化结果存储字典
anova_results_by_cluster = {}

# 需要比较的 cluster 列表
clusters_to_compare = [1, 2, 3]

# 遍历除了'name'和'cluster'外的所有列
for column in data.columns.drop(['name', 'cluster']):
    # 存储当前列的比较结果
    column_results = {}
    
    # 对每个需要比较的 cluster 执行 ANOVA
    for compare_cluster in clusters_to_compare:
        # 提取 cluster = 4 和当前比较 cluster 的数据
        group1 = data[data['cluster'] == 4][column]
        group2 = data[data['cluster'] == compare_cluster][column]

        # 执行 ANOVA
        f_val, p_val = f_oneway(group1, group2)
        column_results[f'cluster_4_vs_{compare_cluster}'] = {'F-value': f_val, 'p-value': p_val}
    
    # 将当前列的结果存入总字典
    anova_results_by_cluster[column] = column_results

# 将结果转换为更方便阅读的形式
anova_df = pd.DataFrame.from_dict({(i, j): anova_results_by_cluster[i][j] 
                                   for i in anova_results_by_cluster.keys() 
                                   for j in anova_results_by_cluster[i].keys()},
                                  orient='index')

# 可以将结果保存为 CSV 文件
anova_df.to_csv(f'{root_path}/results/anova/48h/cluster_4_comparisons.csv', encoding="utf-8-sig")


#### 两两之间的差异

In [23]:
from scipy.stats import f_oneway
import pandas as pd
import itertools

# 假设 df_filtered 是你的数据框
data = df_filtered

# 初始化结果存储字典
anova_results_by_cluster = {}

# 获取所有 unique 的 cluster 值
clusters = data['cluster'].unique()

# 遍历除了 'name' 和 'cluster' 外的所有列
for column in data.columns.drop(['name', 'cluster']):
    # 存储当前列的比较结果
    column_results = {}
    
    # 对每个可能的 cluster 组合执行 ANOVA
    for (cluster1, cluster2) in itertools.combinations(clusters, 2):
        # 提取 cluster1 和 cluster2 的数据
        group1 = data[data['cluster'] == cluster1][column]
        group2 = data[data['cluster'] == cluster2][column]

        # 执行 ANOVA
        f_val, p_val = f_oneway(group1, group2)
        column_results[f'cluster_{cluster1}_vs_{cluster2}'] = {'F-value': f_val, 'p-value': p_val}
    
    # 将当前列的结果存入总字典
    anova_results_by_cluster[column] = column_results

# 将结果转换为更方便阅读的形式
anova_df = pd.DataFrame.from_dict({(i, j): anova_results_by_cluster[i][j] 
                                   for i in anova_results_by_cluster.keys() 
                                   for j in anova_results_by_cluster[i].keys()},
                                  orient='index')

# 可以将结果保存为 CSV 文件
anova_df.to_csv(f'{root_path}/results/anova/48h/all_clusters_comparisons.csv', encoding="utf-8-sig")
